## Feast Client with RBAC
### Kubernetes RBAC Authorization

## Feast Role-Based Access Control (RBAC) in Kubernetes  

Feast **Role-Based Access Control (RBAC)** in Kubernetes supports authentication both **inside a Kubernetes pod** and for **external clients** using the `LOCAL_K8S_TOKEN` environment variable.  


### Inside a Kubernetes Pod
Feast automatically retrieves the Kubernetes ServiceAccount token from:
```
/var/run/secrets/kubernetes.io/serviceaccount/token
```
This means:
- No manual configuration is needed inside a pod.
- The token is mounted automatically and used for authentication.
- Developer just need create the binding with role and service account accordingly.
- Code Reference:  
[Feast Kubernetes Auth Client Manager (Pod Token Usage)](https://github.com/feast-dev/feast/blob/master/sdk/python/feast/permissions/client/kubernetes_auth_client_manager.py#L15) 
-  Using a service account from a pod   [Example](https://github.com/feast-dev/feast/blob/master/examples/rbac-remote/client/k8s/)

### Outside a Kubernetes Pod (External Clients & Local Testing)
 
If running Feast outside of Kubernetes, authentication requires setting the token manually to the environment variable `LOCAL_K8S_TOKEN` :
```sh
export LOCAL_K8S_TOKEN="your-service-account-token"
```

For more details, refer the user guide:  [Kubernetes RBAC Authorization](https://docs.feast.dev/master/getting-started/components/authz_manager#kubernetes-rbac-authorization)  


## Test Cases
| User Type       | ServiceAccount               | RoleBinding Assigned | Expected Behavior in output                                |
|----------------|-----------------------------|----------------------|------------------------------------------------------------|
| **Read-Only**  | `feast-user-sa`              | `feast-reader`       | Can **read** from the feature store, but **cannot write**. |
| **Unauthorized** | `feast-unauthorized-user-sa` | _None_               | **Access should be denied** in `test.py`.                  |
| **Admin**      | `feast-admin-sa`             | `feast-writer`       | Can **read and write** feature store data.                 |

###  Feature Store settings

In [1]:
!cat client/feature_store.yaml

project: feast_rbac
provider: local
offline_store:
    host: localhost
    type: remote
    port: 8081
online_store:
    path: http://localhost:8082
    type: remote
registry:
    path: localhost:8083
    registry_type: remote
auth:
    type: kubernetes
entity_key_serialization_version: 3


**The Operator client feature store ConfigMap** containing the `feature_store.yaml `settings. We can retrieve it and port froward to local as we are testing locally.

In [34]:
!kubectl get configmap feast-sample-kubernetes-auth-client -n feast -o jsonpath='{.data.feature_store\.yaml}' 

project: feast_rbac
provider: local
offline_store:
    host: feast-sample-kubernetes-auth-offline.feast.svc.cluster.local
    type: remote
    port: 80
online_store:
    path: http://feast-sample-kubernetes-auth-online.feast.svc.cluster.local:80
    type: remote
registry:
    path: feast-sample-kubernetes-auth-registry.feast.svc.cluster.local:80
    registry_type: remote
auth:
    type: kubernetes
entity_key_serialization_version: 3


### The function below is executed to support the preparation of client testing.

Run Port Forwarding for All Services for local testing 

In [2]:
import subprocess

# Define services and their local ports
services = {
    "offline_store": ("feast-sample-kubernetes-auth-offline", 8081),
    "online_store": ("feast-sample-kubernetes-auth-online", 8082),
    "registry": ("feast-sample-kubernetes-auth-registry", 8083),
}

# Start port-forwarding for each service
port_forward_processes = {}
for name, (service, local_port) in services.items():
    cmd = f"kubectl port-forward svc/{service} -n feast {local_port}:80"
    process = subprocess.Popen(cmd, shell=True)
    port_forward_processes[name] = process
    print(f"Port forwarding {service} -> localhost:{local_port}")

Port forwarding feast-sample-kubernetes-auth-offline -> localhost:8081
Port forwarding feast-sample-kubernetes-auth-online -> localhost:8082
Port forwarding feast-sample-kubernetes-auth-registry -> localhost:8083


Function to retrieve a Kubernetes service account token and set it as an environment variable

In [3]:
import subprocess
import os

def get_k8s_token(service_account):
    namespace = "feast"

    if not service_account:
        raise ValueError("Service account name is required.")

    result = subprocess.run(
        ["kubectl", "create", "token", service_account, "-n", namespace],
        capture_output=True, text=True, check=True
    )

    token = result.stdout.strip()

    if not token:
        return None  # Silently return None if token retrieval fails

    os.environ["LOCAL_K8S_TOKEN"] = token
    return "Token Retrieved: ***** (hidden for security)"


**Generating training data**. The following test functions were copied from the `test_workflow.py` template but we added `try` blocks to print only 
the relevant error messages, since we expect to receive errors from the permission enforcement modules.

In [51]:
from feast import FeatureStore
from feast.data_source import PushMode
from datetime import datetime
import pandas as pd

# Initialize Feature Store
store = FeatureStore(repo_path="client")

def fetch_historical_features_entity_df(store: FeatureStore, for_batch_scoring: bool):
    """Fetch historical features for training or batch scoring."""
    try:
        entity_df = pd.DataFrame.from_dict(
            {
                "driver_id": [1001, 1002, 1003],
                "event_timestamp": [
                    datetime(2021, 4, 12, 10, 59, 42),
                    datetime(2021, 4, 12, 8, 12, 10),
                    datetime(2021, 4, 12, 16, 40, 26),
                ],
                "label_driver_reported_satisfaction": [1, 5, 3],
                "val_to_add": [1, 2, 3],
                "val_to_add_2": [10, 20, 30],
            }
        )
        if for_batch_scoring:
            entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)

        training_df = store.get_historical_features(
            entity_df=entity_df,
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",
                "transformed_conv_rate:conv_rate_plus_val1",
                "transformed_conv_rate:conv_rate_plus_val2",
            ],
        ).to_df()
        print(f"Successfully fetched {'batch scoring' if for_batch_scoring else 'training'} historical features:\n", training_df.head())

    except PermissionError:
        print("\n*** PERMISSION DENIED *** Cannot fetch historical features.")
    except Exception as e:
        print(f"Unexpected error while fetching historical features: {e}")

def fetch_online_features(store: FeatureStore, source: str = ""):
    """Fetch online features from the feature store."""
    try:
        entity_rows = [
            {
                "driver_id": 1001,
                "val_to_add": 1000,
                "val_to_add_2": 2000,
            },
            {
                "driver_id": 1002,
                "val_to_add": 1001,
                "val_to_add_2": 2002,
            },
        ]
        if source == "feature_service":
            features_to_fetch = store.get_feature_service("driver_activity_v1")
        elif source == "push":
            features_to_fetch = store.get_feature_service("driver_activity_v3")
        else:
            features_to_fetch = [
                "driver_hourly_stats:acc_rate",
                "transformed_conv_rate:conv_rate_plus_val1",
                "transformed_conv_rate:conv_rate_plus_val2",
            ]

        returned_features = store.get_online_features(
            features=features_to_fetch,
            entity_rows=entity_rows,
        ).to_dict()

        print(f"Successfully fetched online features {'via feature service' if source else 'directly'}:\n")
        for key, value in sorted(returned_features.items()):
            print(f"{key} : {value}")

    except PermissionError:
        print("\n*** PERMISSION DENIED *** Cannot fetch online features.")
    except Exception as e:
        print(f"Unexpected error while fetching online features: {e}")

def check_permissions():
    """Check user role, test various Feast operations,."""

    feature_views = []

    # Step 1: List feature views
    print("\n--- List feature views ---")
    try:
        feature_views = store.list_feature_views()
        if not feature_views:
            print("No feature views found. You might not have access or they haven't been created.")
            has_feature_view_access = False
        else:
            print(f"Successfully listed {len(feature_views)} feature views:")
            for fv in feature_views:
                print(f"  - {fv.name}")

    except PermissionError:
        print("\n*** PERMISSION DENIED *** Cannot list feature views.")
        has_feature_view_access = False
    except Exception as e:
        print(f"Unexpected error listing feature views: {e}")
        has_feature_view_access = False

        # Step 2: Fetch Historical Features
    print("\n--- Fetching Historical Features for Training ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=False)

    print("\n--- Fetching Historical Features for Batch Scoring ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=True)

    # Step 3: Apply Feature Store
    print("\n--- Write to Feature Store ---")
    try:
        store.apply(feature_views)
        print("User has write access to the feature store.")
    except PermissionError:
        print("\n*** PERMISSION DENIED *** User lacks permission to modify the feature store.")
    except Exception as e:
        print(f"Unexpected error testing write access: {e}")

    # Step 4: Fetch Online Features
    print("\n--- Fetching Online Features ---")
    fetch_online_features(store)

    print("\n--- Fetching Online Features via Feature Service ---")
    fetch_online_features(store, source="feature_service")

    print("\n--- Fetching Online Features via Push Source ---")
    fetch_online_features(store, source="push")

    print("\n--- Performing Push Source ---")
    # Step 5: Simulate Event Push (Streaming Ingestion)
    try:
        event_df = pd.DataFrame.from_dict(
            {
                "driver_id": [1001],
                "event_timestamp": [datetime.now()],
                "created": [datetime.now()],
                "conv_rate": [1.0],
                "acc_rate": [1.0],
                "avg_daily_trips": [1000],
            }
        )
        store.push("driver_stats_push_source", event_df, to=PushMode.ONLINE_AND_OFFLINE)
        print("Successfully pushed a test event.")
    except PermissionError:
        print("\n*** PERMISSION DENIED *** Cannot push event (no write access).")
    except Exception as e:
        print(f"Unexpected error while pushing event: {e}")


### Test Read-Only Feast User 
**Step 1: Set the Token**

In [48]:
get_k8s_token("feast-user-sa")

'Token Retrieved: ***** (hidden for security)'

**Step 2: Test misc functions from offline, online, materialize_incremental, and others**

In [53]:
# Run the permission check function
check_permissions()



--- List feature views ---
Successfully listed 2 feature views:
  - driver_hourly_stats
  - driver_hourly_stats_fresh

--- Fetching Historical Features for Training ---
Handling connection for 8081
Successfully fetched training historical features:
    driver_id           event_timestamp  label_driver_reported_satisfaction  \
0       1001 2021-04-12 10:59:42+00:00                                   1   
1       1002 2021-04-12 08:12:10+00:00                                   5   
2       1003 2021-04-12 16:40:26+00:00                                   3   

   val_to_add  val_to_add_2  conv_rate  acc_rate  avg_daily_trips  \
0           1            10   0.677818  0.453707              193   
1           2            20   0.328160  0.900565              929   
2           3            30   0.787191  0.958963              571   

   conv_rate_plus_val1  conv_rate_plus_val2  
0             1.677818            10.677818  
1             2.328160            20.328160  
2             3.78719

### Test Unauthorized Feast User 

In [54]:
# Retrieve and store the token
get_k8s_token("feast-unauthorized-user-sa")

'Token Retrieved: ***** (hidden for security)'

In [55]:
check_permissions()


--- List feature views ---
No feature views found. You might not have access or they haven't been created.

--- Fetching Historical Features for Training ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Fetching Historical Features for Batch Scoring ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Write to Feature Store ---

*** PERMISSION DENIED *** User lacks permission to modify the feature store.

--- Fetching Online Features ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Fetching Online Features via Feature Service ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Fetching Online Features via Push Source ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Performing Push Source ---
Unexpected error while pushing event: Unable to find push source 'driver_stats_push_source'.


## Test Admin Feast User

In [56]:
# Retrieve and store the token
get_k8s_token("feast-admin-sa")

'Token Retrieved: ***** (hidden for security)'

In [57]:
check_permissions()


--- List feature views ---
Successfully listed 2 feature views:
  - driver_hourly_stats
  - driver_hourly_stats_fresh

--- Fetching Historical Features for Training ---
Handling connection for 8081
Successfully fetched training historical features:
    driver_id           event_timestamp  label_driver_reported_satisfaction  \
0       1001 2021-04-12 10:59:42+00:00                                   1   
1       1002 2021-04-12 08:12:10+00:00                                   5   
2       1003 2021-04-12 16:40:26+00:00                                   3   

   val_to_add  val_to_add_2  conv_rate  acc_rate  avg_daily_trips  \
0           1            10   0.677818  0.453707              193   
1           2            20   0.328160  0.900565              929   
2           3            30   0.787191  0.958963              571   

   conv_rate_plus_val1  conv_rate_plus_val2  
0             1.677818            10.677818  
1             2.328160            20.328160  
2             3.78719

 **Note:**
**Currently, remote materialization not available in Feast when using the Remote Client**
**Workaround: Consider using running it from pod like**
  
 `kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- bash -c 'feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")`


Terminate the process

In [25]:
for name, process in port_forward_processes.items():
    process.terminate()
    print(f"Stopped port forwarding for {name}")

Stopped port forwarding for offline_store
Stopped port forwarding for online_store
Stopped port forwarding for registry


[Next: Uninstall the Operator and all Feast objects](./03-uninstall.ipynb)